通过使用深度学习框架来简洁地实现3.2中的<u>线性回归模型</u>。

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

[tensor([[ 1.7397, -0.4311],
         [ 0.5734, -0.1188],
         [-0.0656,  0.8856],
         [ 0.7020, -0.0350],
         [-1.6101, -1.0367],
         [ 1.1669,  0.7425],
         [ 1.9434,  0.4760],
         [ 1.9552,  0.0360],
         [-0.2135, -1.0550],
         [ 0.3202,  1.9097]]),
 tensor([[ 9.1407],
         [ 5.7539],
         [ 1.0570],
         [ 5.7332],
         [ 4.5083],
         [ 4.0260],
         [ 6.4810],
         [ 8.0038],
         [ 7.3454],
         [-1.6576]])]

In [7]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [8]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [9]:
loss = nn.MSELoss()

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [11]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000219
epoch 2, loss 0.000092
epoch 3, loss 0.000093


In [12]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0006, -0.0013])
b的估计误差： tensor([-0.0002])


# 总结

## 一、核心思想
利用 PyTorch 的高级 API（torch.utils.data、torch.nn、torch.optim）封装数据处理、模型构建、损失计算、优化器等重复工作，无需手动实现数据迭代器、层逻辑、梯度更新细节，大幅简化代码，同时保持线性回归的核心逻辑（数据→模型→损失→优化）不变，适配复杂模型的扩展需求。
## 二、关键知识点
### 1. 数据集生成与读取（数据流水线）
#### （1）生成合成数据集
- 复用d2l.synthetic_data函数，基于真实参数true_w、true_b生成带噪声的线性数据集（与从零实现逻辑一致）；
- 输出：features（特征矩阵，形状(样本数, 特征数)）、labels（标签向量，形状(样本数, 1)）。
#### （2）框架内置数据迭代器
- 核心 API：torch.utils.data.TensorDataset + torch.utils.data.DataLoader
 - TensorDataset(*data_arrays)：将特征和标签按样本对齐，打包为数据集对象（无需手动处理索引对齐）；
 - DataLoader(dataset, batch_size, shuffle=is_train)：按批量大小加载数据，shuffle=True训练时打乱样本（提升训练稳定性，无需手动写随机洗牌逻辑）；
- 优势：支持批量加载、并行处理，效率远高于手动实现的data_iter。
### 2. 模型定义（神经网络层封装）
#### （1）核心组件
- torch.nn（简称nn）：PyTorch 内置神经网络模块，包含预定义层、损失函数等；
- nn.Sequential：层的容器，按顺序串联多个层（输入→第一层→第二层→...→输出），适配后续多层模型扩展（即使单一层也建议使用，统一流水线）；
- nn.Linear(in_features, out_features)：全连接层（线性层），封装了权重weight和偏置bias的存储与计算：
 - 参数：in_features（输入特征数）、out_features（输出特征数）；
 - 功能：自动实现y = X @ w + b的线性运算（无需手动写矩阵乘法）。
#### （2）模型实例化

In [14]:
net = nn.Sequential(nn.Linear(2, 1))  # 输入2个特征，输出1个标量（回归任务）

### 3. 模型参数初始化

**目标**：权重从均值 0、标准差 0.01 的正态分布采样，偏置初始化为 0；

**关键操作**：通过层的data属性访问参数（避免触发计算图），使用_结尾的 in-place 方法修改参数：

net[0].weight.data.normal_(0, 0.01)：权重正态分布初始化；

net[0].bias.data.fill_(0)：偏置填充为 0；

**逻辑**：net[0]访问Sequential中第一个（也是唯一）层，weight和bias是层的内置参数张量。

### 4. 损失函数（框架预定义）

**核心 API**：nn.MSELoss()（均方误差损失，对应平方 L2 范数）；

**特性**：默认返回所有样本损失的平均值（无需手动求和 / 平均，与从零实现的squared_loss逻辑一致，但更简洁）；

**用法**：直接传入模型预测值net(X)和真实标签y，自动计算损失。

### 5. 优化算法（框架预定义）

**核心 API**：torch.optim.SGD（小批量随机梯度下降）；

**参数说明**：

第一个参数：net.parameters()：自动获取模型中所有可训练参数（权重w、偏置b），无需手动传入；

第二个参数：lr（学习率），控制参数更新步长；

**优势**：框架自动实现梯度下降的参数更新逻辑，无需手动写param -= lr * grad / batch_size。

### 6. 训练流程（核心步骤不变，细节简化）
#### （1）迭代逻辑

迭代周期（num_epochs）：遍历整个数据集的次数；

批量迭代：通过data_iter自动获取小批量(X, y)，无需手动处理索引。

#### （2）单批量训练步骤

前向传播：l = loss(net(X), y) → 生成预测并计算损失；

梯度清零：trainer.zero_grad() → 避免梯度累积（框架默认不自动清零，必须手动调用）；

反向传播：l.backward() → 自动计算所有可训练参数的梯度；

参数更新：trainer.step() → 按优化器（SGD）规则更新参数（无需手动操作梯度）。

#### （3）训练监控

每个迭代周期结束后，计算全量数据的损失，评估模型拟合效果。

### 7. 模型评估

访问训练后的参数：通过net[0].weight.data（权重）、net[0].bias.data（偏置）获取；

对比真实参数与估计参数，验证模型拟合精度（与从零实现的评估逻辑一致）。


| 功能        | 从零实现（手动）       | 简洁实现（PyTorch API）          |
|-------------|------------------------|----------------------------------|
| 数据迭代器  | 自定义 data_iter       | TensorDataset + DataLoader       |
| 线性模型    | 自定义 linreg 函数     | nn.Sequential(nn.Linear)         |
| 损失函数    | 自定义 squared_loss    | nn.MSELoss()                     |
| 优化器      | 自定义 sgd 函数        | torch.optim.SGD()                |
| 参数初始化  | 手动创建 w、b          | weight.data.normal_() 等         |
| 梯度更新    | 手动 param -= lr*grad  | trainer.step()                   |

20260110_2306